In [2]:
import os
import geopandas as gpd
import pandas as pd

# os.environ["USE_PYGEOS"] = os.environ.get("USE_PYGEOS", "0")

local_crs = 32636
# path to data
example_data_path = "./data"

# load data required for blocks cutter
no_development = gpd.read_file(os.path.join(example_data_path, "no_development_pzz.geojson")).to_crs(local_crs)
landuse = gpd.read_file(os.path.join(example_data_path, "landuse_zone_pzz.geojson")).to_crs(local_crs)
okn = gpd.read_file(os.path.join(example_data_path, "petropavlovsraya_delete.geojson")).to_crs(local_crs)

In [2]:
no_development = pd.concat([no_development[["geometry"]], okn[["geometry"]]], axis=0)

In [10]:
from masterplan_tools.method import BlocksCutter
from masterplan_tools.method.blocks import BlocksCutterParameters, BlocksCutterGeometries

city_geometry = gpd.read_parquet(os.path.join(example_data_path, "city_geometry.parquet")).to_crs(local_crs)
water_geometry = gpd.read_parquet(os.path.join(example_data_path, "water_geometry.parquet")).to_crs(local_crs)
roads_geometry = gpd.read_parquet(os.path.join(example_data_path, "roads_geometry.parquet")).to_crs(local_crs)
railways_geometry = gpd.read_parquet(os.path.join(example_data_path, "railways_geometry.parquet")).to_crs(local_crs)
nature_geometry = gpd.read_parquet(os.path.join(example_data_path, "nature_geometry_boundaries.parquet")).to_crs(local_crs)

In [13]:
geoms = BlocksCutterGeometries(
  city=city_geometry,
  water=water_geometry,
  roads=roads_geometry,
  railways=railways_geometry,
  nature=nature_geometry,
  no_development=no_development,
  landuse=landuse
)

In [15]:
from masterplan_tools.method.blocks import BlocksCutter

blocks = BlocksCutter(geometries = geoms).get_blocks()

AttributeError: 'BlocksCutterParameters' object has no attribute 'roads_buffer'

In [ ]:
from masterplan_tools.method.blocks import LuFilter

blocks = LuFilter(blocks).filter_lu()

In [ ]:
from masterplan_tools.Blocks_getter import BlocksClusterization

buildings_geom = gpd.read_feather("/home/gk/code/blocks_ysc/spb_buildings_geom.feather")
blocks = BlocksClusterization(blocks, buildings_geom).run()
blocks = blocks[blocks["landuse"].isin(["new_buildings", "buildings"])]